# Experiments for Efficient Mixed Integer Linear Programming Approaches to Dynamic Path Restoration

In [1]:
import json
import math
import os
import re

from functools import reduce

import sys
import shutil

from IPython.display import display_markdown

TERMINAL_SIZE = shutil.get_terminal_size(fallback=(120, 50)).columns

import numpy as np
import pandas as pd
from collections import defaultdict
from dataclasses import dataclass

nb_dir = os.path.split(os.getcwd())[0]
#print(nb_dir[:-len("/jupyter")])
if nb_dir not in sys.path:
    sys.path.append(nb_dir[:-len("/jupyter")])


In [2]:
from lib.config import Config
from lib.optical_network import OpticalNetwork
from algorithms.gready_search import GreedyAlgorithm
from config.paths import DATA_SETS_DIR, TOPOLOGY_DIR, BENCHMARK_DIR

In [3]:
@dataclass(frozen=True)
class Experiment:
    net: str
    mod_name: str

    @property
    def name(self):
        return f"{self.net}_1d1_{self.mod_name}_n1_221" 
        
    def __str__(self):
        return f"{self.net}-{self.mod_name}"
    
    def __repr__(self):
        return f"Experiment({self.net}, {self.mod_name})"

In [4]:
#%%time
net_names = ["NSFNET", "USNET"]
NSFNET = net_names[0]
NSFNET, USNET = net_names
BPSK, QPSK = "BPSK", "QPSK" 
dist_dict = {"BPSK": 5000, "QPSK": 2500, "8-QAM": 1250, "16-QAM": 625}
configs = {(exp := Experiment(net, mod_name)): Config(f"{exp.name}_mixed_all.json") 
           for net in net_names 
           for mod_name in dist_dict.keys()}

In [5]:
confs = [
"USNET_1d1_BPSK_n1_221_mixed_all.json",
"USNET_1d1_BPSK_n1_221_b23_mixed_all.json",
"USNET_1d1_QPSK_n1_221_mixed_all.json",
"USNET_1d1_QPSK_n1_221_b23_mixed_all.json",
"USNET_1d1_8-QAM_n1_221_mixed_all.json",
"USNET_1d1_8-QAM_n1_221_b31_mixed_all.json",
"NSFNET_1d1_BPSK_n1_221_mixed_all.json",
"NSFNET_1d1_BPSK_n1_221_b5_mixed_all.json",
"NSFNET_1d1_QPSK_n1_221_mixed_all.json",
"NSFNET_1d1_QPSK_n1_221_b19_mixed_all.json"]

In [6]:
confs_scip = [
 'USNET_1d1_BPSK_n1_221_mixed_all_scip.json',
 'USNET_1d1_BPSK_n1_221_b23_mixed_all_scip.json',
 'USNET_1d1_QPSK_n1_221_mixed_all_scip.json',
 'USNET_1d1_QPSK_n1_221_b23_mixed_all_scip.json',
 'USNET_1d1_8-QAM_n1_221_mixed_all_scip.json',
 'USNET_1d1_8-QAM_n1_221_b31_mixed_all_scip.json',
 'NSFNET_1d1_BPSK_n1_221_mixed_all_scip.json',
 'NSFNET_1d1_BPSK_n1_221_b5_mixed_all_scip.json',
 'NSFNET_1d1_QPSK_n1_221_mixed_all_scip.json',
 'NSFNET_1d1_QPSK_n1_221_b19_mixed_all_scip.json']

In [7]:
configs = {}
for conf in confs:
    net, mod_name, suffix = re.split('_1d1_|_n1_', conf)
    m = re.search("b([0-9]+)", suffix)
    configs[(Experiment(net,mod_name), int(m.group(1)) if m else None)] = Config(conf)

In [8]:
networks = {key: OpticalNetwork(config.data_set_dir_name)
            for key, config in configs.items()}    

In [9]:
configs_scip = {}
for conf in confs_scip:
    net, mod_name, suffix = re.split('_1d1_|_n1_', conf)
    m = re.search("b([0-9]+)", suffix)
    configs_scip[(Experiment(net,mod_name), int(m.group(1)) if m else None)] = Config(conf)

In [10]:
networks_scip = {key: OpticalNetwork(config.data_set_dir_name)
            for key, config in configs_scip.items()}    

In [11]:
milp_algs = ["basic_dlc@mixed", "reach_color_trim@mixed",
             "reach_color_trim@mixed_no_trimming"]

In [12]:
exp = Experiment(NSFNET, BPSK)

In [13]:
def exp_mean(config, column="total_time", fmt='{:,.2f}', exclude_links=[]):    
    _df = (config 
           . df_results
               . query("algorithm == @milp_algs and dropped_link_id != @exclude_links")
               . groupby("algorithm") [column] . mean() . map(fmt.format)
               . to_frame())
    _df.index.name = None
    return _df

def exp_max(config, column="total_time", fmt='{:,.2f}', exclude_links=[]):
    _df = (config 
           . df_results
               . query("algorithm == @milp_algs and dropped_link_id != @exclude_links")
               . groupby("algorithm") [column] . max() . map(fmt.format)
               . to_frame())
    _df.index.name = None
    return _df

def exp_min(config, column="total_time", fmt='{:,.2f}', exclude_links=[]):
    _df = (config 
           . df_results
               . query("algorithm == @milp_algs and dropped_link_id != @exclude_links")
               . groupby("algorithm") [column] . min() . map(fmt.format)
               . to_frame())
    _df.index.name = None
    return _df

In [14]:
rename_algs = {"basic_dlc@mixed": "Basic", 
               "reach_color_trim@mixed": "Trimmed",
               "reach_color_trim@mixed_no_trimming": "NoTrim"}

In [15]:
def summary_df(map_conf, _configs=configs, _networks=networks):
    _df = pd.concat([map_conf(config).transpose() for (exp, lnk), config in _configs.items()])
    _df.index = pd.MultiIndex.from_tuples([(exp.net, exp.mod_name, lnk)
                                       for exp, lnk in _configs.keys()],
                                      names=['net', 'mod', "link"])
    _df["#tests"] = [len(_networks[(Experiment(net, mod), (None if math.isnan(lnk) else lnk))].mixed_links)
                 for net, mod, lnk in _df.index]
    _df = _df.rename(rename_algs, axis=1)[["#tests", "Basic", "NoTrim", "Trimmed"]]
    return _df 

In [16]:
tables_gen = {"Max Vars": lambda df: exp_max(df, "vars", "{:,.2E}"), 
              "Mean Vars": lambda df: exp_mean(df, "vars", "{:,.2E}"),
              "Min Vars": lambda df: exp_min(df, "vars", "{:,.2E}")}
for header, func in tables_gen.items():
    _df = summary_df(func)
    display_markdown(f"### {header}", raw=True)
    display(_df)

### Max Vars

#tests     Basic    NoTrim   Trimmed
net    mod   link                                      
USNET  BPSK  NaN       41  2.28E+05  1.24E+05  1.86E+04
             23        40  2.76E+05  1.48E+05  1.42E+04
       QPSK  NaN       37  1.75E+05  1.15E+05  7.59E+03
             23        36  1.71E+05  1.10E+05  7.59E+03
       8-QAM NaN        2  2.28E+05  2.18E+05  1.63E+04
             31         3  2.23E+05  2.07E+05  1.63E+04
NSFNET BPSK  NaN       21  1.11E+05  6.45E+04  1.42E+04
             5         20  1.18E+05  6.65E+04  9.87E+03
       QPSK  NaN       12  6.38E+04  5.01E+04  5.17E+03
             19        11  7.68E+04  5.88E+04  5.17E+03

### Mean Vars

#tests     Basic    NoTrim   Trimmed
net    mod   link                                      
USNET  BPSK  NaN       41  1.46E+05  7.66E+04  9.02E+03
             23        40  1.48E+05  7.57E+04  7.10E+03
       QPSK  NaN       37  9.95E+04  6.38E+04  2.80E+03
             23        36  9.73E+04  6.24E+04  2.70E+03
       8-QAM NaN        2  2.28E+05  2.18E+05  1.36E+04
             31         3  2.14E+05  2.02E+05  5.44E+03
NSFNET BPSK  NaN       21  6.40E+04  3.49E+04  6.97E+03
             5         20  6.77E+04  3.44E+04  4.12E+03
       QPSK  NaN       12  4.26E+04  3.25E+04  2.30E+03
             19        11  4.65E+04  3.47E+04  1.79E+03

### Min Vars

#tests     Basic    NoTrim   Trimmed
net    mod   link                                      
USNET  BPSK  NaN       41  6.05E+04  3.06E+04  1.74E+03
             23        40  5.90E+04  2.89E+04  1.02E+03
       QPSK  NaN       37  6.72E+03  4.18E+03  6.80E+01
             23        36  6.56E+03  4.08E+03  1.24E+02
       8-QAM NaN        2  2.28E+05  2.18E+05  1.09E+04
             31         3  2.10E+05  2.00E+05  0.00E+00
NSFNET BPSK  NaN       21  3.70E+04  1.84E+04  2.48E+03
             5         20  3.52E+04  1.64E+04  9.48E+02
       QPSK  NaN       12  2.69E+04  2.02E+04  9.42E+02
             19        11  2.56E+04  1.86E+04  0.00E+00

In [17]:
display_markdown(f"# CBC", raw=True)
tables_gen = {"Max Time": exp_max, "Mean Time": exp_mean, "Min Time": exp_min}
for header, func in tables_gen.items():
    display_markdown(f"### {header}", raw=True)
    display(summary_df(func))    

# CBC

### Max Time

#tests   Basic  NoTrim Trimmed
net    mod   link                                
USNET  BPSK  NaN       41   37.57   21.87    3.38
             23        40  526.03  519.38    1.90
       QPSK  NaN       37   49.49   35.57    2.50
             23        36  111.83   41.35    3.32
       8-QAM NaN        2  806.94  540.09    2.09
             31         3  560.44  501.64    2.40
NSFNET BPSK  NaN       21   17.97    8.68    1.95
             5         20  172.71  323.98    2.06
       QPSK  NaN       12  110.09  644.92    1.05
             19        11  108.15  515.37    1.00

### Mean Time

#tests   Basic  NoTrim Trimmed
net    mod   link                                
USNET  BPSK  NaN       41   15.60    9.92    1.69
             23        40   48.77   33.29    0.78
       QPSK  NaN       37   14.48   11.07    0.83
             23        36   19.60   11.61    0.84
       8-QAM NaN        2  670.17  526.72    1.96
             31         3  259.80  193.16    1.11
NSFNET BPSK  NaN       21    6.95    4.81    1.07
             5         20   16.40   19.34    0.54
       QPSK  NaN       12   25.54   61.85    0.61
             19        11   14.84   51.92    0.41

### Min Time

#tests   Basic  NoTrim Trimmed
net    mod   link                                
USNET  BPSK  NaN       41    5.18    3.47    0.48
             23        40    4.16    2.22    0.23
       QPSK  NaN       37    0.63    0.50    0.12
             23        36    0.68    0.54    0.14
       8-QAM NaN        2  533.41  513.35    1.84
             31         3   85.76   29.52    0.46
NSFNET BPSK  NaN       21    3.17    1.53    0.39
             5         20    2.84    1.09    0.13
       QPSK  NaN       12    4.24    2.70    0.34
             19        11    2.77    2.62    0.16

In [18]:
display_markdown(f"# SCIP", raw=True)
tables_gen = {"Max Time": exp_max, "Mean Time": exp_mean, "Min Time": exp_min}
for header, func in tables_gen.items():
    display_markdown(f"### {header}", raw=True)
    display(summary_df(func,configs_scip, networks_scip))    

# SCIP

### Max Time

#tests   Basic  NoTrim Trimmed
net    mod   link                                
USNET  BPSK  NaN       41  114.90  100.99    8.77
             23        40  485.98  505.77    5.02
       QPSK  NaN       37   74.61   78.35    2.49
             23        36  122.59  116.76    2.28
       8-QAM NaN        2  657.28  780.99   19.17
             31         3  748.57  745.98   24.68
NSFNET BPSK  NaN       21  594.92   93.56    3.96
             5         20   58.68   45.77    1.88
       QPSK  NaN       12  106.07   74.52    0.77
             19        11   73.69   60.37    0.75

### Mean Time

#tests   Basic  NoTrim Trimmed
net    mod   link                                
USNET  BPSK  NaN       41   39.23   34.43    2.72
             23        40   51.36   47.23    1.36
       QPSK  NaN       37   28.23   26.27    0.75
             23        36   31.07   28.53    0.68
       8-QAM NaN        2  592.51  658.69   17.85
             31         3  566.57  361.64    9.20
NSFNET BPSK  NaN       21  116.06   32.29    1.77
             5         20   13.64   11.24    0.44
       QPSK  NaN       12   47.47   26.49    0.44
             19        11   29.46   25.59    0.33

### Min Time

#tests   Basic  NoTrim Trimmed
net    mod   link                                
USNET  BPSK  NaN       41   10.12    6.64    0.39
             23        40    8.22    6.26    0.23
       QPSK  NaN       37    1.26    0.77    0.09
             23        36    0.81    0.64    0.10
       8-QAM NaN        2  527.74  536.38   16.54
             31         3  410.39   45.84    1.46
NSFNET BPSK  NaN       21    6.71    5.19    0.38
             5         20    1.82    0.87    0.09
       QPSK  NaN       12   15.32    7.02    0.24
             19        11    9.58    4.12    0.13

In [19]:
def small_summary(config, header="Time", column="total_time",
                  fmt='{:,.2f}', excluded_links=[]): 
    _mean = exp_mean(config, column, fmt, excluded_links).transpose()
    _max = exp_max(config, column, fmt, excluded_links).transpose()
    _min = exp_min(config, column, fmt, excluded_links).transpose()
    _mean.index = ["mean"]
    _max.index = ["max"]
    _min.index = ["min"]
    _stat = pd.concat([_max, _mean,  _min])
    _stat.index.name = header
    return _stat.rename(rename_algs, axis=1)[["Basic", "NoTrim", "Trimmed"]]

In [20]:
def group_columns(df):
    time_cols = [name for name in df.columns if name.endswith("_time")]    
    alg_cols = ["answer", "comment"]
    
    d_rename_dict = {"number_of_broken_demands": "initially", 
                     "pruning_recovered": "recovered", "pruning_recovered_mixed": "mixed",
                     "long_unsatisfiable_demands": "always_unsat",
                     "unsatisfiable_demands": "unsatisfiable"
                                             }
    demand_rename = {key: val for key,val in d_rename_dict.items() # "demands_in_cut": "in_cut"
                    if key in df.columns}    
    demand_cols = list(demand_rename.keys())
    df_time = pd.concat({"Time": df[time_cols].rename(columns={name: name.removesuffix("_time") for name in time_cols})}, axis=1)

    df_demand = pd.concat({"Broken Demands": df[demand_cols].rename(columns=demand_rename)}, axis=1)
    if ("Broken Demands", "recovered") or ("Broken Demands", "unsatisfiable") in df_demand.columns:
        diff, zeros  = np.zeros(len(df_demand)), np.zeros(len(df_demand))
        diff += df_demand[("Broken Demands", "recovered")].fillna(0) if ("Broken Demands", "recovered") in df_demand else zeros
        diff += df_demand[("Broken Demands", "unsatisfiable")].fillna(0) if ("Broken Demands", "unsatisfiable") in df_demand else zeros            
        df_demand[("Broken Demands", "left")] = df_demand[("Broken Demands", "initially")] - diff
    df_demand = df_demand.astype("Int64")
    
    if ("Broken Demands", "always_unsat") in df_demand.columns:
        df_demand[("Broken Demands", "unsat")] = df_demand[("Broken Demands", "unsatisfiable")].fillna(0)
        df_demand[("Broken Demands", "unsat")] -= df_demand[("Broken Demands", "always_unsat")].fillna(0)
        df_demand.rename(columns={"left": "sat", "recovered": "always_sat"}, inplace=True)
    cut_rename = {"demands_in_cut": "dems", "links_in_cut": "links", "cut_edges": "edges"}
    df_cut = pd.concat({"Cut": df[cut_rename.keys()].rename(columns=cut_rename)}, axis=1).astype("Int64") if "demands_in_cut" in df.columns \
             else pd.DataFrame()    
    cut_columns = list(cut_rename.keys()) if "demands_in_cut" in df.columns else []
    
    df_alg = pd.concat({"Algorithm": df[alg_cols]}, axis=1)
    df_meta = pd.concat({"Meta": df.drop(["solution_computed"]+time_cols+demand_cols+alg_cols + cut_columns, axis=1)}, axis=1)
    
    _df = pd.concat([df_alg, df_demand, df_cut, df_meta, df_time], axis=1)
        
    _df[("Meta", "vars")] = _df[("Meta", "vars")].map("{:,.2E}".format)
    
    _df = _df.droplevel(["net", "dist"])
    _df.index = pd.MultiIndex.from_tuples(
        [(lnk, rename_algs[alg]) for lnk, alg in _df.index], names=['link', 'algorithm'])

    
    
    if ("Broken Demands", "always_unsat") in df_demand.columns:    
        #_df.columns = [_df.columns.get_level_values(0), np.repeat("", _df.shape[1])]
        _df.columns = pd.MultiIndex.from_tuples(map(lambda x: (x[0], '', x[1]), _df.columns))
        _df[("Demands", "initially", "broken")] = df_demand[("Broken Demands", "initially")]        
        _df[("Demands", "always", "sat")] = df_demand[("Broken Demands", "always_sat")]        
        _df[("Demands", "always", "unsat")] = df_demand[("Broken Demands", "always_unsat")]
        _df[("Demands", "result", "sat")] = df_demand[("Broken Demands", "sat")]        
        _df[("Demands", "result", "unsat")] = df_demand[("Broken Demands", "unsat")]        
        

        _df.drop(columns=["Broken Demands"], inplace=True)


    
        return _df[["Algorithm", "Demands", "Meta", "Time"]]
    return _df

In [21]:
display_markdown(f"# Hard Test Cases :: CBC", raw=True)
for (exp, lnk), config in configs.items():
    __df = (config._df_results(validate=False)                          
                          . query("algorithm == @milp_algs and total_time >= 100"))
    if len(__df)==0:
        continue
    _links = [lnk for lnk, _ in group_columns(__df).index]
    lnk_str = "" if lnk is None else f"-{lnk}"    
    display_markdown(f"## {exp}{lnk_str}", raw=True)
    #print((exp, lnk))
    if len(_links) < len(networks[(exp, lnk)].mixed_links):
        display_markdown(f"### Statistics with excluded hard test cases", raw=True)
        #display(small_summary(config, "Vars","vars", "{:,.2E}", _links))
        display(small_summary(config, excluded_links=_links))    
    else:
        display_markdown(f"### All links are hard, so we do not update the statictics", 
                         raw=True)
        display(small_summary(config))   
        
    
    with pd.option_context('display.max_rows', None):                    
        _df = ((group_columns(
                config._df_results(validate=False)
                    . query("algorithm == @milp_algs") 
                            ).drop(columns=[("Broken Demands", "left")]))   
               . query("link == @_links")).sort_index()
        display(_df)

# Hard Test Cases :: CBC

## USNET-BPSK-23

### Statistics with excluded hard test cases

,Basic,NoTrim,Trimmed
Time,,,
max,97.07,21.25,1.90
mean,21.77,7.71,0.75
min,4.16,2.22,0.23


Algorithm                                               \
                  answer                                      comment   
link algorithm                                                          
30   Basic          <NA>  TerminationCondition=intermediateNonInteger   
     NoTrim         <NA>  TerminationCondition=intermediateNonInteger   
     Trimmed       False                            Infeasible model.   
32   Basic          <NA>  TerminationCondition=intermediateNonInteger   
     NoTrim         <NA>  TerminationCondition=intermediateNonInteger   
     Trimmed       False                            Infeasible model.   
40   Basic         False                            Infeasible model.   
     NoTrim        False                            Infeasible model.   
     Trimmed       False                            Infeasible model.   

               Broken Demands      Meta        Time                         
                    initially      vars       total init_model milp_solver  
link algorithm                                                              
30   Basic                 38  2.49E+05  510.650894   7.644625  502.979717  
     NoTrim                38  1.30E+05  519.383958   3.698638  515.667421  
     Trimmed               38  1.36E+04    1.454116   0.978346    0.461984  
32   Basic                 28  1.84E+05  526.025439   7.107922  518.892680  
     NoTrim                28  9.48E+04  514.616816   2.563118  512.035544  
     Trimmed               28  1.29E+04    1.187734   0.644915    0.528083  
40   Basic                 28  1.84E+05  108.544389   4.196317  104.333438  
     NoTrim                28  9.44E+04   12.095266   2.618700    9.460689  
     Trimmed               28  9.62E+03    0.929523   0.597969    0.316788

...

Algorithm                                               \
                  answer                                      comment   
link algorithm                                                          
1    Basic          True                         MILP Solution found.   
     NoTrim         True                         MILP Solution found.   
     Trimmed        True                         MILP Solution found.   
6    Basic          True                         MILP Solution found.   
     NoTrim         <NA>  TerminationCondition=intermediateNonInteger   
     Trimmed        True                         MILP Solution found.   

               Broken Demands      Meta        Time                         
                    initially      vars       total init_model milp_solver  
link algorithm                                                              
1    Basic                 18  6.05E+04  110.088938   2.122554  107.660549  
     NoTrim                18  4.74E+04   47.556855   2.015708   45.247494  
     Trimmed               18  5.17E+03    1.049253   0.474053    0.524662  
6    Basic                 19  6.38E+04  106.226817   2.675926  103.205915  
     NoTrim                19  5.01E+04  644.921483   3.110555  641.803181  
     Trimmed               19  4.21E+03    0.895373   0.491302    0.354835

## NSFNET-QPSK-19

### Statistics with excluded hard test cases

,Basic,NoTrim,Trimmed
Time,,,
max,9.51,21.56,1.00
mean,5.50,5.58,0.37
min,2.77,2.62,0.16


Algorithm                                               \
                  answer                                      comment   
link algorithm                                                          
6    Basic          True                         MILP Solution found.   
     NoTrim         <NA>  TerminationCondition=intermediateNonInteger   
     Trimmed        True                         MILP Solution found.   

               Broken Demands      Meta        Time                         
                    initially      vars       total init_model milp_solver  
link algorithm                                                              
6    Basic                 19  6.08E+04  108.148777   2.465347  105.386686  
     NoTrim                19  4.63E+04  515.367168   1.920008  513.439457  
     Trimmed               19  4.21E+03    0.800766   0.443823    0.318720

In [22]:
display_markdown(f"# Hard Test Cases :: SCIP", raw=True)
for (exp, lnk), config in configs_scip.items():
    __df = (config._df_results(validate=False)                          
                          . query("algorithm == @milp_algs and total_time >= 100"))
    if len(__df)==0:
        continue
    _links = [lnk for lnk, _ in group_columns(__df).index]
    lnk_str = "" if lnk is None else f"-{lnk}"    
    display_markdown(f"## {exp}{lnk_str}", raw=True)
    #print((exp, lnk))
    if len(_links) < len(networks[(exp, lnk)].mixed_links):
        display_markdown(f"### Statistics with excluded hard test cases", raw=True)
        #display(small_summary(config, "Vars","vars", "{:,.2E}", _links))
        display(small_summary(config, excluded_links=_links))    
    else:
        display_markdown(f"### All links are hard, so we do not update the statictics", 
                         raw=True)
        display(small_summary(config))   
        
    
    with pd.option_context('display.max_rows', None):                    
        _df = ((group_columns(
                config._df_results(validate=False)
                    . query("algorithm == @milp_algs") 
                            ).drop(columns=[("Broken Demands", "left")]))   
               . query("link == @_links")).sort_index()
        display(_df)

# Hard Test Cases :: SCIP

## USNET-BPSK

### Statistics with excluded hard test cases

,Basic,NoTrim,Trimmed
Time,,,
max,89.53,80.93,8.77
mean,37.34,32.77,2.73
min,10.12,6.64,0.39


Algorithm                       Broken Demands      Meta  \
                  answer               comment      initially      vars   
link algorithm                                                            
10   Basic          True  MILP Solution found.             24  1.61E+05   
     NoTrim         True  MILP Solution found.             24  8.47E+04   
     Trimmed        True  MILP Solution found.             24  1.02E+04   

                      Time                         
                     total init_model milp_solver  
link algorithm                                     
10   Basic      114.896097   4.761306  109.560241  
     NoTrim     100.988362   2.819681   97.823889  
     Trimmed      2.601055   0.584639    1.942913

...

Algorithm                             Broken Demands      Meta  \
                  answer                     comment      initially      vars   
link algorithm                                                                  
5    Basic          <NA>  TerminationCondition=other             17  5.71E+04   
     NoTrim         <NA>  TerminationCondition=other             17  3.04E+04   
     Trimmed        True        MILP Solution found.             17  6.41E+03   
7    Basic          True        MILP Solution found.             30  1.01E+05   
     NoTrim         True        MILP Solution found.             30  5.78E+04   
     Trimmed        True        MILP Solution found.             30  1.42E+04   
8    Basic          True        MILP Solution found.             22  7.39E+04   
     NoTrim         True        MILP Solution found.             22  4.04E+04   
     Trimmed        True        MILP Solution found.             22  7.60E+03   
10   Basic          True        MILP Solution found.             33  1.11E+05   
     NoTrim         True        MILP Solution found.             33  6.45E+04   
     Trimmed        True        MILP Solution found.             33  1.31E+04   
11   Basic          True        MILP Solution found.             29  9.74E+04   
     NoTrim         True        MILP Solution found.             29  5.57E+04   
     Trimmed        True        MILP Solution found.             29  1.07E+04   
12   Basic          <NA>         Solver's Time Limit             25  8.40E+04   
     NoTrim         <NA>  TerminationCondition=other             25  4.62E+04   
     Trimmed        True        MILP Solution found.             25  9.15E+03   
14   Basic          True        MILP Solution found.             16  5.38E+04   
     NoTrim         True        MILP Solution found.             16  2.73E+04   
     Trimmed        <NA>  TerminationCondition=other             16  5.98E+03   

                      Time                         
                     total init_model milp_solver  
link algorithm                                     
5    Basic      142.885170   5.163382  137.696294  
     NoTrim      36.169919   0.584685   35.577865  
     Trimmed      2.364688   0.328248    1.995674  
7    Basic      370.369249   8.041741  361.253578  
     NoTrim      93.561101   1.755028   91.572573  
     Trimmed      3.347811   0.864511    2.402205  
8    Basic      130.981595   4.939285  125.309461  
     NoTrim      31.185818   1.133740   29.885885  
     Trimmed      1.403967   0.331160    1.011855  
10   Basic      327.586653   8.191695  318.313298  
     NoTrim      90.246524   2.037965   87.957878  
     Trimmed      3.962630   0.462749    3.420893  
11   Basic      151.033480   4.864824  145.753559  
     NoTrim      87.407349   1.300395   85.881785  
     Trimmed      2.903850   0.764222    2.062860  
12   Basic      594.922887   2.264186  592.651000  
     NoTrim      55.093617   1.435611   53.650512  
     Trimmed      1.609795   0.416265    1.125202  
14   Basic      158.022523   1.600982  156.228821  
     NoTrim      13.864343   1.035385   12.704758  
     Trimmed      1.898104   0.249172    1.640392

## NSFNET-QPSK

### Statistics with excluded hard test cases

,Basic,NoTrim,Trimmed
Time,,,
max,87.81,74.52,0.71
mean,42.14,27.54,0.41
min,15.32,7.02,0.24


Algorithm                       Broken Demands      Meta  \
                  answer               comment      initially      vars   
link algorithm                                                            
1    Basic          True  MILP Solution found.             18  6.05E+04   
     NoTrim         True  MILP Solution found.             18  4.74E+04   
     Trimmed        True  MILP Solution found.             18  5.17E+03   

                      Time                         
                     total init_model milp_solver  
link algorithm                                     
1    Basic      106.071128   1.428753  104.429957  
     NoTrim      14.844925   1.322852   13.347687  
     Trimmed      0.768823   0.328102    0.403886

In [23]:
display_markdown(f"# Detailed Results", raw=True)
for (exp, lnk), config in configs.items():
    config_scip = configs_scip[(exp,lnk)]
    lnk_str = "" if lnk is None else f"-{lnk}"    
    display_markdown(f"## {exp}{lnk_str}", raw=True)
    #print((exp, lnk))
    test_cases = len(networks[(exp, lnk)].mixed_links)    
    satisfiable = len(config._df_results(validate=False)
                      . query("algorithm == 'reach_color_trim@mixed' and answer==True"))
    display_markdown(f"**Number of test cases**: {test_cases}",
                     raw=True)
    display_markdown(f"**Satisfiable**: {satisfiable} **Unatisfiable**: {test_cases - satisfiable}",
                     raw=True)
    display(small_summary(config, "Vars","vars", "{:,.2E}"))
    display_markdown(f"### CBC", raw=True)
    display(small_summary(config))    
    display_markdown(f"### SCIP", raw=True)
    display(small_summary( config_scip))  
    display_markdown(f"### CBC", raw=True)
    with pd.option_context('display.max_rows', None):                    
        _df = (group_columns(config._df_results(validate=False)
                             . query("algorithm == @milp_algs"))
               . drop(columns=[("Broken Demands", "left")])).sort_index()        
        display(_df)
    display_markdown(f"### SCIP", raw=True)
    display(small_summary( config_scip))    
    with pd.option_context('display.max_rows', None):                    
        _df = (group_columns(config_scip._df_results(validate=False)
                             . query("algorithm == @milp_algs"))
               . drop(columns=[("Broken Demands", "left")])).sort_index()        
        display(_df)

# Detailed Results

## USNET-BPSK

**Number of test cases**: 41

**Satisfiable**: 41 **Unatisfiable**: 0

,Basic,NoTrim,Trimmed
Vars,,,
max,2.28E+05,1.24E+05,1.86E+04
mean,1.46E+05,7.66E+04,9.02E+03
min,6.05E+04,3.06E+04,1.74E+03


...

### CBC

Algorithm                                               \
                  answer                                      comment   
link algorithm                                                          
0    Basic          True                         MILP Solution found.   
     NoTrim         True                         MILP Solution found.   
     Trimmed        True                         MILP Solution found.   
1    Basic          True                         MILP Solution found.   
     NoTrim         True                         MILP Solution found.   
     Trimmed        True                         MILP Solution found.   
2    Basic          True                         MILP Solution found.   
     NoTrim         True                         MILP Solution found.   
     Trimmed        True                         MILP Solution found.   
3    Basic          True                         MILP Solution found.   
     NoTrim         True                         MILP Solution found.   
     Trimmed        True                         MILP Solution found.   
6    Basic          True                         MILP Solution found.   
     NoTrim         <NA>  TerminationCondition=intermediateNonInteger   
     Trimmed        True                         MILP Solution found.   
8    Basic          True                         MILP Solution found.   
     NoTrim         True                         MILP Solution found.   
     Trimmed        True                         MILP Solution found.   
12   Basic         False                            Infeasible model.   
     NoTrim        False                            Infeasible model.   
     Trimmed       False                   Initially infeasible model   
13   Basic         False                            Infeasible model.   
     NoTrim        False                            Infeasible model.   
     Trimmed       False                            Infeasible model.   
16   Basic         False                            Infeasible model.   
     NoTrim        False                            Infeasible model.   
     Trimmed       False                   Initially infeasible model   
17   Basic         False                            Infeasible model.   
     NoTrim        False                            Infeasible model.   
     Trimmed       False                   Initially infeasible model   
18   Basic          True                         MILP Solution found.   
     NoTrim         True                         MILP Solution found.   
     Trimmed        True                         MILP Solution found.   

               Broken Demands      Meta        Time                         
                    initially      vars       total init_model milp_solver  
link algorithm                                                              
0    Basic                  9  2.88E+04    2.767629   0.910719    1.737690  
     NoTrim                 9  2.07E+04    3.133390   1.566189    1.457043  
     Trimmed                9  1.11E+03    0.321701   0.195618    0.109695  
1    Basic                 18  5.76E+04    7.852599   1.733007    5.879023  
     NoTrim                18  4.37E+04   21.562051   1.483198   19.852147  
     Trimmed               18  5.17E+03    0.997349   0.483279    0.469686  
2    Basic                 16  5.12E+04    5.764417   1.483722    4.058762  
     NoTrim                16  3.79E+04    5.020082   1.559270    3.266003  
     Trimmed               16  3.95E+03    0.629267   0.297907    0.296850  
3    Basic                 10  3.20E+04    3.459638   1.266320    2.056316  
     NoTrim                10  2.31E+04    2.811162   1.084090    1.571464  
     Trimmed               10  1.23E+03    0.304275   0.190525    0.094209  
6    Basic                 19  6.08E+04  108.148777   2.465347  105.386686  
     NoTrim                19  4.63E+04  515.367168   1.920008  513.439457  
     Trimmed               19  4.21E+03    0.800766   0.443823    0.318720  


### SCIP

,Basic,NoTrim,Trimmed
Time,,,
max,73.69,60.37,0.75
mean,29.46,25.59,0.33
min,9.58,4.12,0.13


Algorithm                             Broken Demands      Meta  \
                  answer                     comment      initially      vars   
link algorithm                                                                  
0    Basic          True        MILP Solution found.              9  2.88E+04   
     NoTrim         True        MILP Solution found.              9  2.07E+04   
     Trimmed        True        MILP Solution found.              9  1.11E+03   
1    Basic          True        MILP Solution found.             18  5.76E+04   
     NoTrim         True        MILP Solution found.             18  4.37E+04   
     Trimmed        True        MILP Solution found.             18  5.17E+03   
2    Basic          True        MILP Solution found.             16  5.12E+04   
     NoTrim         True        MILP Solution found.             16  3.79E+04   
     Trimmed        True        MILP Solution found.             16  3.95E+03   
3    Basic          True        MILP Solution found.             10  3.20E+04   
     NoTrim         True        MILP Solution found.             10  2.31E+04   
     Trimmed        True        MILP Solution found.             10  1.23E+03   
6    Basic          True        MILP Solution found.             19  6.08E+04   
     NoTrim         True        MILP Solution found.             19  4.63E+04   
     Trimmed        True        MILP Solution found.             19  4.21E+03   
8    Basic          True        MILP Solution found.             10  3.20E+04   
     NoTrim         True        MILP Solution found.             10  2.34E+04   
     Trimmed        True        MILP Solution found.             10  1.49E+03   
12   Basic         False           Infeasible model.             14  4.48E+04   
     NoTrim        False           Infeasible model.             14  3.33E+04   
     Trimmed       False  Initially infeasible model             14  1.96E+02   
13   Basic         False           Infeasible model.             12  3.84E+04   
     NoTrim        False           Infeasible model.             12  2.84E+04   
     Trimmed       False           Infeasible model.             12  1.03E+03   
16   Basic         False           Infeasible model.             20  6.40E+04   
     NoTrim        False           Infeasible model.             20  4.80E+04   
     Trimmed       False  Initially infeasible model             20  6.36E+02   
17   Basic         False           Infeasible model.             24  7.68E+04   
     NoTrim        False           Infeasible model.             24  5.88E+04   
     Trimmed       False  Initially infeasible model             24  0.00E+00   
18   Basic          True        MILP Solution found.              8  2.56E+04   
     NoTrim         True        MILP Solution found.              8  1.86E+04   
     Trimmed        True        MILP Solution found.              8  6.22E+02   

                     Time                         
                    total init_model milp_solver  
link algorithm                                    
0    Basic       9.576801   0.771978    8.702783  
     NoTrim      8.748835   1.082450    7.467160  
     Trimmed     0.240531   0.155134    0.069904  
1    Basic      51.027780   1.502922   49.326527  
     NoTrim     53.652482   1.148208   52.337991  
     Trimmed     0.746420   0.309423    0.402408  
2    Basic      38.191571   1.275506   36.738711  
     NoTrim     40.182953   1.211550   38.828882  
     Trimmed     0.519063   0.231287    0.257404  
3    Basic      11.716032   1.047071   10.557390  
     NoTrim      9.811036   0.823149    8.902801  
     Trimmed     0.242915   0.153996    0.072780  
6    Basic      66.609707   1.914636   64.486282  
     NoTrim     59.858596   1.423577   58.241158  
     Trimmed     0.634902   0.326267    0.277785  
8    Basic      15.730943   0.812698   14.803045  
     NoTrim     11.494195   0.745816   10.659818  
     Trimmed     0.281187   0.156650    0.105946  
12   Basic      11.055162  